In [0]:
#df = spark.read.option('multiline' 'true').json('dbfs:/FileStore/tables/Sample_data.json')

# Read multiline json file
df = spark.read.option("multiline","true") \
      .json("dbfs:/FileStore/tables/Sample_data.json")    

display(df)

Employee
"List(List(Manager, List(List(List(List(ep, 46044403, 1, MP)), 46555002, 1)), 46066601), List(Supervisor, List(List(List(List(ep, 56044403, 1, MP), List(fs, 56044000, 0, D)), 5605501, 1)), 56555000))"


In [0]:
##Define Python function to flatten deeply nested JSON file

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

#Flatten array of structs and structs
def flatten(df):
   # compute Complex Fields (Lists and Structs) in Schema   
   complex_fields = dict([(field.name, field.dataType)
                             for field in df.schema.fields
                             if type(field.dataType) == ArrayType or  type(field.dataType) == StructType])
   while len(complex_fields)!=0:
      col_name=list(complex_fields.keys())[0]
      print ("Processing :"+col_name+" Type : "+str(type(complex_fields[col_name])))
    
      # if StructType then convert all sub element to columns.
      # i.e. flatten structs
      if (type(complex_fields[col_name]) == StructType):
         expanded = [col(col_name+'.'+k).alias(col_name+'_'+k) for k in [ n.name for n in  complex_fields[col_name]]]
         df=df.select("*", *expanded).drop(col_name)
    
      # if ArrayType then add the Array Elements as Rows using the explode function
      # i.e. explode Arrays
      elif (type(complex_fields[col_name]) == ArrayType):    
         df=df.withColumn(col_name,explode_outer(col_name))
    
      # recompute remaining Complex Fields in Schema       
      complex_fields = dict([(field.name, field.dataType)
                             for field in df.schema.fields
                             if type(field.dataType) == ArrayType or  type(field.dataType) == StructType])
   return df


In [0]:
##Applying Flattening Function and Display flattened data

df_flatten = flatten(df)
display(df_flatten)

Processing :Employee Type : <class 'pyspark.sql.types.ArrayType'>
Processing :Employee Type : <class 'pyspark.sql.types.StructType'>
Processing :Employee_attribute Type : <class 'pyspark.sql.types.ArrayType'>
Processing :Employee_attribute Type : <class 'pyspark.sql.types.StructType'>
Processing :Employee_attribute_Department Type : <class 'pyspark.sql.types.ArrayType'>
Processing :Employee_attribute_Department Type : <class 'pyspark.sql.types.StructType'>


Employee_Destination,Employee_emp_id,Employee_attribute_Parent_id,Employee_attribute_status_flag,Employee_attribute_Department_Code,Employee_attribute_Department_Dept_id,Employee_attribute_Department_dept_flag,Employee_attribute_Department_dept_type
Manager,46066601,46555002,1,ep,46044403,1,MP
Supervisor,56555000,5605501,1,ep,56044403,1,MP
Supervisor,56555000,5605501,1,fs,56044000,0,D
